# I. SINGAPORE'S WEATHER DATA: 1983 - 2019 (SEPT)
This dataset is one of the few detailed, multi-year ones that I've been able to find in the public domain in Singapore. I believe it can be useful for data science students in this part of the world who are looking to test their skills on a local dataset, or build a personal project.

I'll be using this dataset for small projects on data visualisation (see notebook 2.0_visualisation_cch), time series analysis and machine learning. Ping me on LinkedIn or Twitter if you do something interesting with this set of data:

Twitter: @chinhon

LinkedIn: https://www.linkedin.com/in/chuachinhon/

### UPDATE: 
Temperature records for July-Sept 2019 have been added. 

## FILE ORGANISATION:
The original data files, as downloaded from the [Singapore Met Office](http://www.weather.gov.sg/climate-historical-daily/) and Data.gov.sg, are in the raw folder. The files are mostly clean, save for some missing values for mean and max wind speed. I've lightly processed the files and saved the output to the data folder so that I can call them up easily for future data projects.

You can make a different version of the dataset by concating the raw files over a different time frame, or with more elaborate feature engineering.

What you'll find in the raw folder:
- 438 CSV files containing daily weather data for Singapore from 1983 - 2019 (June)

- a "monthly_data" sub-folder containing monthly average data for rainfall, maximum and mean temperatures.

The files in the data folder have been processed by the code below.

In [1]:
import glob
import pandas as pd

# 1. DAILY WEATHER DATA 

In [2]:
# Combining the separate CSV files into one
raw = pd.concat(
    [pd.read_csv(f) for f in glob.glob("../raw/*.csv")], ignore_index=True
)

In [3]:
# Adding a datetime col in the year-month-day format
raw["Date"] = pd.to_datetime(
    raw["Year"].astype(str)
    + "-"
    + raw["Month"].astype(str)
    + "-"
    + raw["Day"].astype(str)
)

In [4]:
raw["Month_Name"] = raw["Date"].dt.month_name()
raw["Quarter"] = raw["Date"].dt.quarter

In [5]:
# Converting values in the Max/Mean Wind into numeric data type
raw["Max Wind Speed (km/h)"] = pd.to_numeric(
    raw["Max Wind Speed (km/h)"], errors="coerce"
)
raw["Mean Wind Speed (km/h)"] = pd.to_numeric(
    raw["Mean Wind Speed (km/h)"], errors="coerce"
)

In [6]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13422 entries, 0 to 13421
Data columns (total 16 columns):
Station                          13422 non-null object
Year                             13422 non-null int64
Month                            13422 non-null int64
Day                              13422 non-null int64
Daily Rainfall Total (mm)        13422 non-null float64
Highest 30 Min Rainfall (mm)     13422 non-null object
Highest 60 Min Rainfall (mm)     13422 non-null object
Highest 120 Min Rainfall (mm)    13422 non-null object
Mean Temperature (°C)            13422 non-null float64
Maximum Temperature (°C)         13422 non-null float64
Minimum Temperature (°C)         13422 non-null float64
Mean Wind Speed (km/h)           13412 non-null float64
Max Wind Speed (km/h)            13411 non-null float64
Date                             13422 non-null datetime64[ns]
Month_Name                       13422 non-null object
Quarter                          13422 non-null int64
d

#### Fill the missing entries in Mean Wind Speed and Max Wind Speed columns with mean values of both cols

In [7]:
raw["Max Wind Speed (km/h)"] = raw["Max Wind Speed (km/h)"].fillna(
    raw["Max Wind Speed (km/h)"].mean()
)
raw["Mean Wind Speed (km/h)"] = raw["Mean Wind Speed (km/h)"].fillna(
    raw["Mean Wind Speed (km/h)"].mean()
)

In [8]:
# Dropping cols that I won't need for visualisation or modelling
raw = raw.drop(
    columns=[
        "Station",
        "Highest 30 Min Rainfall (mm)",
        "Highest 60 Min Rainfall (mm)",
        "Highest 120 Min Rainfall (mm)",
    ]
)

In [9]:
# Slight rearrangement of cols for clarity
cols = [
    "Date",
    "Year",
    "Month",
    "Month_Name",
    "Quarter",
    "Day",
    "Daily Rainfall Total (mm)",
    "Mean Temperature (°C)",
    "Maximum Temperature (°C)",
    "Minimum Temperature (°C)",
    "Mean Wind Speed (km/h)",
    "Max Wind Speed (km/h)",
]

In [10]:
weather = raw[cols].copy()

In [11]:
weather = weather.sort_values('Date', ascending=False)

In [12]:
weather.info()
# no null values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13422 entries, 1400 to 4385
Data columns (total 12 columns):
Date                         13422 non-null datetime64[ns]
Year                         13422 non-null int64
Month                        13422 non-null int64
Month_Name                   13422 non-null object
Quarter                      13422 non-null int64
Day                          13422 non-null int64
Daily Rainfall Total (mm)    13422 non-null float64
Mean Temperature (°C)        13422 non-null float64
Maximum Temperature (°C)     13422 non-null float64
Minimum Temperature (°C)     13422 non-null float64
Mean Wind Speed (km/h)       13422 non-null float64
Max Wind Speed (km/h)        13422 non-null float64
dtypes: datetime64[ns](1), float64(6), int64(4), object(1)
memory usage: 1.3+ MB


In [13]:
weather.columns

Index(['Date', 'Year', 'Month', 'Month_Name', 'Quarter', 'Day',
       'Daily Rainfall Total (mm)', 'Mean Temperature (°C)',
       'Maximum Temperature (°C)', 'Minimum Temperature (°C)',
       'Mean Wind Speed (km/h)', 'Max Wind Speed (km/h)'],
      dtype='object')

In [14]:
weather.describe()
# The Daily Rainfall cols have some obvious outliers. But let's deal with that later, as and when required

,Year,Month,Quarter,Day,Daily Rainfall Total (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h)
count,13422.000000,13422.000000,13422.000000,13422.000000,13422.000000,13422.000000,13422.000000,13422.000000,13422.000000,13422.000000
mean,2000.875950,6.492326,2.498361,15.728654,5.815281,27.667151,31.518246,24.902637,7.445743,34.056342
std,10.608927,3.439891,1.114069,8.799939,14.413763,1.176099,1.571405,1.268748,3.480851,8.037197
min,1983.000000,1.000000,1.000000,1.000000,0.000000,22.800000,23.600000,20.200000,0.200000,4.700000
25%,1992.000000,4.000000,2.000000,8.000000,0.000000,26.900000,30.800000,24.000000,4.800000,28.800000
50%,2001.000000,7.000000,3.000000,16.000000,0.000000,27.700000,31.800000,24.900000,6.800000,33.100000
75%,2010.000000,9.000000,3.000000,23.000000,4.400000,28.600000,32.500000,25.800000,9.700000,38.200000
max,2019.000000,12.000000,4.000000,31.000000,216.200000,30.900000,36.000000,29.100000,22.200000,90.700000


In [15]:
weather.head()

,Date,Year,Month,Month_Name,Quarter,Day,Daily Rainfall Total (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h)
1400,2019-09-30,2019,9,September,3,30,1.4,27.5,31.3,25.1,6.8,34.2
1399,2019-09-29,2019,9,September,3,29,0.0,28.4,33.3,26.5,6.8,41.4
1398,2019-09-28,2019,9,September,3,28,0.0,28.9,34.3,26.0,7.6,29.5
1397,2019-09-27,2019,9,September,3,27,0.0,29.2,34.1,26.8,9.7,30.6
1396,2019-09-26,2019,9,September,3,26,0.0,28.5,33.5,25.4,9.0,31.0


In [16]:
# weather.to_csv('../data/weather.csv', index=False)

## 2. MONTHLY DATA
Here, I'll do some light processing of the monthly average data for rainfall, maximum and mean temperatures. They are in the raw folder's "monthly_data" sub-folder.

### 2.1 MONTHLY RAINFALL RECORDS

In [17]:
monthly_rain = pd.read_csv('../raw/monthly_data/monthly_rain.csv')

In [18]:
monthly_rain["month"] = pd.to_datetime(monthly_rain["month"])
monthly_rain["year"] = monthly_rain["month"].dt.year
monthly_rain["month"] = monthly_rain["month"].dt.month

In [19]:
monthly_rain = monthly_rain.rename(columns = {"year": "Year", 
                                              "month": "Month", 
                                              "total_rainfall": "Total_Monthly_Rainfall (mm)"})

In [20]:
# For consistency with the daily records, I'll start with entries from 1983 for the monthly datasets as well 
cols_rain = ["Total_Monthly_Rainfall (mm)", "Year", "Month"]
monthly_rain = monthly_rain[cols_rain].copy()
monthly_rain = monthly_rain[monthly_rain["Year"] >= 1983]

In [21]:
#monthly_rain.to_csv('../data/rain_monthly.csv', index=False)

In [22]:
monthly_rain.tail()

,Total_Monthly_Rainfall (mm),Year,Month
445,31.6,2019,2
446,72.2,2019,3
447,174.8,2019,4
448,69.0,2019,5
449,173.8,2019,6


### 2.2 MONTHLY MEAN TEMPERATURES

In [23]:
mean_temp = pd.read_csv('../raw/monthly_data/monthly_temp_mean.csv')

In [24]:
mean_temp["month"] = pd.to_datetime(mean_temp["month"])
mean_temp["year"] = mean_temp["month"].dt.year
mean_temp["month"] = mean_temp["month"].dt.month

In [25]:
mean_temp = mean_temp.rename(
    columns={
        "year": "Year",
        "month": "Month",
        "mean_temp": "Mean_Monthly_Temperature (°C)",
    }
)

In [26]:
cols_temp_mean = ["Mean_Monthly_Temperature (°C)", "Year", "Month"]
mean_temp = mean_temp[cols_temp_mean].copy()
mean_temp = mean_temp[mean_temp["Year"] >= 1983]

In [27]:
#mean_temp.to_csv('../data/mean_temp_monthly.csv', index=False)

### 2.3 MONTHLY MAX TEMPERATURES

In [28]:
max_temp = pd.read_csv('../raw/monthly_data/monthly_temp_max.csv')

In [29]:
max_temp["month"] = pd.to_datetime(max_temp["month"])
max_temp["year"] = max_temp["month"].dt.year
max_temp["month"] = max_temp["month"].dt.month

In [30]:
max_temp = max_temp.rename(
    columns={
        "year": "Year",
        "month": "Month",
        "max_temperature": "Max_Monthly_Temperature (°C)",
    }
)

In [31]:
cols_temp_max = ["Max_Monthly_Temperature (°C)", "Year", "Month"]
max_temp = max_temp[cols_temp_max].copy()
max_temp = max_temp[max_temp["Year"] >= 1983]

In [32]:
#max_temp.to_csv('../data/max_temp_monthly.csv', index=False)